In [ ]:
# 🔧 Setup: Run this cell first!
# Check GPU availability and install dependencies

import torch
import sys

# Check GPU
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"✅ GPU available: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    device = torch.device('cpu')
    print("⚠️ No GPU detected. Some cells may run slowly.")
    print("   Go to Runtime → Change runtime type → GPU")

print(f"\n📦 Python {sys.version.split()[0]}")
print(f"🔥 PyTorch {torch.__version__}")

# Set random seeds for reproducibility
import random
import numpy as np

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

print(f"🎲 Random seed set to {SEED}")

%matplotlib inline

# Case Study: Autonomous Pick-and-Place for High-SKU E-Commerce Fulfillment
## Implementation Notebook

# 🤖 AI Teaching Assistant

Need help with this notebook? Open the **AI Teaching Assistant** — it has already read this entire notebook and can help with concepts, code, and exercises.

**[👉 Open AI Teaching Assistant](https://pods.vizuara.ai/courses/world-action-models/practice/0/assistant)**

*Tip: Open it in a separate tab and work through this notebook side-by-side.*


## Business Context

**Company:** NovaPick Robotics (Series B startup, 118 employees, $45M raised)

**Problem:** NovaPick's current robotic pick-and-place system uses hand-engineered grasp planners that work for ~500 pre-programmed SKUs but fail on novel items (34.7% success rate). Their prospective client MegaMart has 83,000+ active SKUs. They need zero-shot generalization to novel objects with >90% pick success rate.

**Approach:** World model (RSSM) for predicting grasp outcomes in latent space + Vision-Language-Action architecture with flow matching for generating smooth robot trajectories conditioned on camera input and language instructions.

**Key Metrics:**

| Metric | Target |
|---|---|
| Pick success rate (novel objects) | > 90% |
| Pick success rate (known objects) | > 97% |
| Cycle time | < 4.0 seconds |
| Inference latency | < 200ms |
| Damage rate | < 0.5% |

## Setup and Imports

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import numpy as np
import matplotlib.pyplot as plt
from typing import Optional
import time

# Check GPU availability
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"GPU available: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device('cpu')
    print("No GPU detected. Go to Runtime > Change runtime type > GPU")

print(f"PyTorch {torch.__version__}")

## 3.1 Data Acquisition Strategy

**Dataset:** A combination of:
- **DROID (Distributed Robot Interaction Dataset):** 76K real-world robot manipulation trajectories with RGB-D images, proprioceptive states, and action labels
- **Google Robot dataset (Open X-Embodiment):** 53K pick-and-place episodes with language annotations

For this notebook, we use a curated subset: 10K episodes from DROID + 5K from Google Robot, pre-processed and hosted on Hugging Face.

**Preprocessing pipeline:**
- Resize RGB images to (3, 224, 224)
- Normalize depth maps to [0, 1] range
- Tokenize language instructions using a frozen CLIP text encoder
- Normalize joint angles to [-1, 1] range
- Segment episodes into overlapping action chunks of 16 timesteps

### TODO: Data Augmentation Pipeline

In [ ]:
def augment_observation(rgb: torch.Tensor, depth: torch.Tensor,
                         language: str) -> tuple[torch.Tensor, torch.Tensor, str]:
    """
    Apply data augmentation to a single observation.

    Args:
        rgb: RGB image tensor of shape (3, 224, 224), values in [0, 1]
        depth: Depth map tensor of shape (1, 224, 224), values in [0, 1]
        language: Language instruction string

    Returns:
        Augmented (rgb, depth, language) tuple

    Requirements:
        1. Apply random color jitter to RGB (brightness +/-0.2, contrast +/-0.2,
           saturation +/-0.2) -- this simulates variable warehouse lighting
        2. Apply the SAME random crop (scale 0.8-1.0) to BOTH rgb and depth --
           spatial correspondence must be preserved
        3. Apply random horizontal flip with p=0.5 to BOTH rgb and depth --
           remember to also flip any spatial terms in the language instruction
           (swap "left" <-> "right")
        4. Add Gaussian noise (std=0.01) to the depth map to simulate sensor noise

    Hints:
        - Use torchvision.transforms for color jitter
        - For paired spatial transforms, generate random parameters first, then
          apply the same params to both modalities
        - For language flipping, use simple string replacement
    """
    # YOUR CODE HERE
    raise NotImplementedError

### Verification: Data Augmentation

In [ ]:
# Test your augmentation
rgb_test = torch.rand(3, 224, 224)
depth_test = torch.rand(1, 224, 224)
lang_test = "pick up the red bottle on the left"

rgb_aug, depth_aug, lang_aug = augment_observation(rgb_test, depth_test, lang_test)

assert rgb_aug.shape == (3, 224, 224), f"RGB shape mismatch: {rgb_aug.shape}"
assert depth_aug.shape == (1, 224, 224), f"Depth shape mismatch: {depth_aug.shape}"
assert rgb_aug.min() >= 0.0, "RGB values must be non-negative"
assert isinstance(lang_aug, str), "Language must be a string"
print("Augmentation tests passed.")

## 3.2 Exploratory Data Analysis

**Key distributions to plot:**
- Distribution of action magnitudes per joint (are some joints used more than others?)
- Distribution of episode lengths (are most episodes short or long?)
- Language instruction length distribution and word frequency analysis
- Depth map quality histogram (what fraction of pixels have valid depth?)
- Gripper mode distribution across the dataset (suction vs. parallel-jaw vs. release)

**Anomalies to investigate:**
- Episodes with near-zero action variance (robot not moving -- possible data collection errors)
- Depth maps with > 30% invalid pixels (sensor occlusion or failure)
- Mismatched language instructions (instruction mentions object not visible in frame)

### TODO: EDA Implementation

In [ ]:
def compute_dataset_statistics(dataloader: DataLoader) -> dict:
    """
    Compute comprehensive statistics over the entire dataset.

    Args:
        dataloader: PyTorch DataLoader yielding (rgb, depth, language, actions,
                    proprioception) tuples

    Returns:
        Dictionary with keys:
            - "action_mean": mean action per joint, shape (7,)
            - "action_std": std action per joint, shape (7,)
            - "episode_lengths": list of episode lengths
            - "depth_valid_fraction": list of valid pixel fractions per sample
            - "num_anomalous_episodes": count of episodes with action std < 0.01

    Hints:
        - Use running statistics (Welford's algorithm) to avoid loading all
          data into memory
        - For depth validity, count pixels where depth > 0.01 (near-zero
          depth indicates invalid measurement)
    """
    # YOUR CODE HERE
    raise NotImplementedError

### Thought Questions: EDA

1. Which joints have the highest action variance? What does this tell you about the most common manipulation motions?
2. What fraction of episodes are "anomalous" (near-zero action)? Should these be filtered or kept? What are the tradeoffs?
3. Is the depth data quality sufficient for learning, or do you need additional preprocessing?

## 3.3 Baseline Approach

Before building the world model + VLA system, we implement a simple baseline to establish a performance floor.

**Baseline: Image-conditioned grasp quality network**

Architecture: frozen ResNet-18 visual encoder + 3-layer MLP that predicts (x, y, z, roll, pitch, yaw, gripper_width) for a single grasp pose. No temporal reasoning, no language conditioning, no world model.

### TODO: Implement Baseline Model

In [ ]:
class GraspBaseline(nn.Module):
    """
    Simple grasp prediction baseline.

    Takes an RGB-D image and predicts a single grasp pose.
    No language input, no temporal reasoning, no world model.

    Architecture:
        1. Frozen ResNet-18 backbone (pretrained on ImageNet) processes the
           RGB image -> 512-dim feature vector
        2. A small CNN (3 conv layers) processes the depth map -> 128-dim
           feature vector
        3. Concatenate visual features (512 + 128 = 640 dims)
        4. MLP: 640 -> 256 -> 128 -> 7 (grasp pose)
        5. Output: (x, y, z, roll, pitch, yaw, gripper_width)

    Hints:
        - Use torchvision.models.resnet18(pretrained=True) and freeze with
          requires_grad_(False)
        - Remove the final FC layer of ResNet; use the 512-dim avgpool output
        - The depth CNN should use (1, 224, 224) input with channels:
          1 -> 32 -> 64 -> 128, kernel size 3, stride 2, with batch norm
        - Apply tanh to the output to constrain predictions to [-1, 1]
    """
    def __init__(self):
        super().__init__()
        # YOUR CODE HERE
        raise NotImplementedError

    def forward(self, rgb: torch.Tensor, depth: torch.Tensor) -> torch.Tensor:
        """
        Args:
            rgb: (B, 3, 224, 224) RGB image
            depth: (B, 1, 224, 224) depth map
        Returns:
            grasp_pose: (B, 7) predicted grasp pose
        """
        # YOUR CODE HERE
        raise NotImplementedError

### Baseline Evaluation

Train this baseline on the dataset using MSE loss against ground-truth grasp poses and evaluate pick success rate in a PyBullet simulation environment.

## 3.4 Model Design

This is the core of the case study. We implement a simplified world model + VLA system inspired by the concepts from the article on World Action Models.

**Architecture overview:**

The system has three main components:
1. **Observation Encoder:** A multimodal encoder that fuses RGB, depth, and language into a unified representation
2. **World Model (RSSM):** A recurrent state-space model that predicts future latent states given actions
3. **Action Generator (Flow Matching):** A conditional flow model that generates action chunks from noise, conditioned on the observation encoding and world model state

### TODO: Implement Observation Encoder

In [ ]:
class ObservationEncoder(nn.Module):
    """
    Multimodal encoder that fuses vision (RGB + depth) and language into a
    single latent representation.

    Architecture:
        1. Visual encoder: Use a frozen DINOv2 ViT-S/14 backbone to encode
           the RGB image into patch tokens. Concatenate depth as a 4th channel
           via a learned linear projection.
           Output: (B, N_patches, 384)

        2. Language encoder: Use a frozen CLIP text encoder to get a language
           embedding.
           Output: (B, 512)

        3. Cross-attention fusion: Use 2 layers of cross-attention where
           language tokens attend to visual patch tokens. This allows the model
           to focus on the part of the image relevant to the instruction.
           Output: (B, 256)

        4. Final projection: Linear layer to produce the observation embedding.
           Output: (B, 256)

    Hints:
        - For DINOv2, use torch.hub.load('facebookresearch/dinov2', 'dinov2_vits14')
        - For CLIP, use clip.load("ViT-B/32") and freeze
        - In cross-attention, language is the query, visual patches are key/value
        - Use nn.MultiheadAttention with 4 heads and embed_dim=384
        - Add a learnable [CLS] token to the language side for pooling
    """
    def __init__(self, obs_embed_dim: int = 256):
        super().__init__()
        # YOUR CODE HERE
        raise NotImplementedError

    def forward(self, rgb: torch.Tensor, depth: torch.Tensor,
                language_tokens: torch.Tensor) -> torch.Tensor:
        """
        Args:
            rgb: (B, 3, 224, 224)
            depth: (B, 1, 224, 224)
            language_tokens: (B, 77) CLIP tokenized text
        Returns:
            obs_embed: (B, 256) fused observation embedding
        """
        # YOUR CODE HERE
        raise NotImplementedError

### Thought Question: Observation Encoder

Why is cross-attention better than simple concatenation for fusing vision and language? Think about what happens when the instruction says "pick up the RED bottle" -- which image patches should receive the most attention?

### TODO: Implement RSSM World Model

In [ ]:
class RSSM(nn.Module):
    """
    Recurrent State-Space Model for world modeling.

    This is the core world model from DreamerV3, adapted for manipulation.
    It maintains both a deterministic recurrent state and a stochastic
    latent state.

    State components:
        - h_t: deterministic state (GRU hidden state), shape (B, 512)
        - z_t: stochastic state (categorical), shape (B, 32, 32)
          -- 32 categorical variables, each with 32 classes
          (flattened to (B, 1024) when needed)

    Sub-modules:
        1. Deterministic transition: GRU that takes [z_{t-1}, a_{t-1}] as
           input and h_{t-1} as hidden state -> h_t

        2. Prior (imagination): MLP that predicts z_t distribution from h_t
           alone (used during dreaming)
           h_t -> MLP(512, 512, 32*32) -> reshape to (B, 32, 32) ->
           softmax over last dim

        3. Posterior (encoder): MLP that predicts z_t distribution from
           [h_t, obs_embed] (used during training with real observations)
           [h_t, obs_embed] -> MLP(768, 512, 32*32) -> reshape -> softmax

        4. Reward predictor: MLP that predicts expected reward from [h_t, z_t]
           [h_t, z_t_flat] -> MLP(1536, 256, 1) -> output scalar

    Hints:
        - Use nn.GRUCell for the deterministic transition
        - For the stochastic state, use straight-through gradients with
          Gumbel-Softmax for differentiable sampling
        - The KL loss between prior and posterior encourages the prior to
          be useful for imagination
        - Start with temperature=1.0 for Gumbel-Softmax
    """
    def __init__(self, obs_embed_dim: int = 256, action_dim: int = 10,
                 det_dim: int = 512, stoch_dim: int = 32,
                 stoch_classes: int = 32):
        super().__init__()
        self.det_dim = det_dim
        self.stoch_dim = stoch_dim
        self.stoch_classes = stoch_classes
        # YOUR CODE HERE
        raise NotImplementedError

    def initial_state(self, batch_size: int, device: torch.device) -> tuple:
        """Return initial (h_0, z_0) with zeros."""
        # YOUR CODE HERE
        raise NotImplementedError

    def observe_step(self, obs_embed: torch.Tensor, action: torch.Tensor,
                     h_prev: torch.Tensor, z_prev: torch.Tensor
                     ) -> tuple[torch.Tensor, torch.Tensor, dict]:
        """
        Single step with real observation (training time).

        Returns: (h_t, z_t, info_dict)
            info_dict contains 'prior_logits', 'posterior_logits' for KL loss
        """
        # YOUR CODE HERE
        raise NotImplementedError

    def imagine_step(self, action: torch.Tensor, h_prev: torch.Tensor,
                     z_prev: torch.Tensor
                     ) -> tuple[torch.Tensor, torch.Tensor]:
        """
        Single step WITHOUT observation (imagination/dreaming).

        Returns: (h_t, z_t) using the prior distribution
        """
        # YOUR CODE HERE
        raise NotImplementedError

    def imagine_trajectory(self, initial_h: torch.Tensor,
                           initial_z: torch.Tensor,
                           actions: torch.Tensor) -> dict:
        """
        Roll out imagination for multiple steps.

        Args:
            initial_h: (B, 512) starting deterministic state
            initial_z: (B, 32, 32) starting stochastic state
            actions: (B, T, 10) sequence of actions to imagine

        Returns:
            dict with 'h_states': (B, T, 512), 'z_states': (B, T, 32, 32),
            'rewards': (B, T)
        """
        # YOUR CODE HERE
        raise NotImplementedError

### Thought Question: RSSM World Model

Why does the RSSM use BOTH deterministic and stochastic states? Consider what would happen with only deterministic (cannot model uncertainty) or only stochastic (loses long-term memory) states.

### TODO: Implement Flow Matching Action Generator

In [ ]:
class FlowMatchingActionHead(nn.Module):
    """
    Generates action chunks via conditional flow matching.

    Instead of predicting actions directly (which produces jerky motions),
    this module learns a velocity field that transforms Gaussian noise into
    smooth action trajectories. This is the same principle used in pi-0.

    Architecture:
        - Input: noisy action chunk a_t, timestep t, conditioning c
          where c = [h, z_flat, obs_embed] from the world model
        - Network: 1D temporal U-Net (or simple MLP with timestep embedding)
          For simplicity, use an MLP variant:
          1. Embed timestep t using sinusoidal embedding -> (B, 128)
          2. Concatenate [a_t_flat, t_embed, c] -> (B, 16*10 + 128 + 1792)
          3. MLP: input_dim -> 1024 -> 512 -> 256 -> 16*10
          4. Reshape output to (B, 16, 10) -- velocity prediction

        - Training: Sample t ~ U(0,1), sample noise a_0 ~ N(0,I),
          interpolate a_t = (1-t)*a_0 + t*a_1 (where a_1 is ground truth),
          predict velocity v = a_1 - a_0

        - Inference: Start from a_0 ~ N(0,I), integrate velocity field
          using Euler method with N=10 steps

    Hints:
        - Sinusoidal embedding: [sin(t*freq), cos(t*freq)] for
          freq in geometric series
        - During training, the target velocity is simply (a_1 - a_0) --
          this is the optimal transport direction
        - Use 10 Euler integration steps at inference for good quality
          vs. speed tradeoff
        - Clip final actions to [-1, 1]
    """
    def __init__(self, action_chunk_size: int = 16, action_dim: int = 10,
                 cond_dim: int = 1792):
        super().__init__()
        self.action_chunk_size = action_chunk_size
        self.action_dim = action_dim
        # YOUR CODE HERE
        raise NotImplementedError

    def forward(self, noisy_actions: torch.Tensor, timestep: torch.Tensor,
                condition: torch.Tensor) -> torch.Tensor:
        """
        Predict velocity field.

        Args:
            noisy_actions: (B, 16, 10) noisy action chunk
            timestep: (B,) diffusion timestep in [0, 1]
            condition: (B, 1792) conditioning from world model
        Returns:
            velocity: (B, 16, 10) predicted velocity
        """
        # YOUR CODE HERE
        raise NotImplementedError

    def sample(self, condition: torch.Tensor, num_steps: int = 10
               ) -> torch.Tensor:
        """
        Generate action chunk from noise via Euler integration.

        Args:
            condition: (B, 1792) conditioning from world model
            num_steps: number of Euler integration steps
        Returns:
            actions: (B, 16, 10) generated action chunk
        """
        # YOUR CODE HERE
        raise NotImplementedError

### Thought Question: Flow Matching

Why is flow matching preferred over a standard diffusion model (DDPM) for action generation? Think about: (a) the straightness of transport paths, (b) the number of inference steps needed, and (c) the smoothness of generated trajectories.

## 3.5 Training Strategy

**Optimizer:** AdamW with weight decay 1e-4

**Learning rate schedule:**
- Warmup: linear from 0 to 3e-4 over first 5K steps
- Cosine decay from 3e-4 to 1e-5 over remaining training
- Total: 200K gradient steps

**Regularization:**
- Dropout 0.1 in MLPs
- Gradient clipping: max norm 100
- KL balancing (DreamerV3 style): 80% free nats, clipped KL loss

**Training loop structure:**
1. Sample a batch of episodes from the dataset
2. Encode observations through the observation encoder
3. Run the RSSM forward with real observations (observe mode) to get latent states
4. Compute world model loss: reconstruction + KL
5. Sample random timesteps and noise for flow matching
6. Compute flow matching loss on action chunks
7. Backpropagate and update all parameters jointly

### TODO: Implement Training Loop

In [ ]:
def train_one_epoch(model: dict, dataloader: DataLoader,
                    optimizer: torch.optim.Optimizer,
                    scheduler: torch.optim.lr_scheduler._LRScheduler,
                    device: torch.device, epoch: int) -> dict:
    """
    Train all model components for one epoch.

    Args:
        model: dict with keys 'encoder', 'rssm', 'flow_head'
               (each an nn.Module)
        dataloader: yields batches of (rgb, depth, language, actions,
                    proprioception, rewards)
        optimizer: shared optimizer for all components
        scheduler: learning rate scheduler
        device: torch device
        epoch: current epoch number

    Returns:
        dict with 'flow_loss', 'wm_loss', 'kl_loss', 'total_loss'
        (averaged over epoch)

    Implementation steps:
        1. For each batch:
           a. Move all tensors to device
           b. Encode observations: obs_embed = encoder(rgb, depth, language)
           c. Initialize RSSM state: h, z = rssm.initial_state(B, device)
           d. Loop over timesteps in the episode:
              - h, z, info = rssm.observe_step(obs_embed[:, t], actions[:, t], h, z)
              - Accumulate KL loss from info['prior_logits'] vs info['posterior_logits']
              - Store h, z for action generation
           e. Compute world model loss (KL + reward prediction)
           f. Sample random t ~ U(0,1) for flow matching
           g. Create noisy actions: a_t = (1-t)*noise + t*ground_truth_actions
           h. Predict velocity: v_pred = flow_head(a_t, t, condition)
           i. Flow loss = MSE(v_pred, ground_truth_actions - noise)
           j. Total loss = flow_loss + 0.5 * wm_loss + 0.1 * grasp_loss
           k. Backprop and optimizer step

        2. Log losses every 100 steps using print or wandb
        3. Step the scheduler after each batch

    Hints:
        - Use torch.distributions.kl_divergence for KL computation between
          categorical distributions
        - Detach the RSSM hidden state at the start of each episode to prevent
          backprop through time explosion
        - Use gradient clipping: torch.nn.utils.clip_grad_norm_(params, 100)
    """
    # YOUR CODE HERE
    raise NotImplementedError

## 3.6 Evaluation

**Quantitative evaluation:**
- Run the trained model in a PyBullet pick-and-place simulation (provided in the notebook)
- Evaluate on two object sets: (a) 50 objects seen during training, (b) 50 held-out novel objects
- Compute all metrics from the evaluation table in Section 2
- Compare against the baseline from Section 3.3

| Metric | Target |
|---|---|
| Pick success rate (novel) | > 90% |
| Pick success rate (known) | > 97% |
| Cycle time | < 4.0 seconds |
| Grasp damage rate | < 0.5% |
| Action smoothness (jerk) | < 500 rad/s^3 |
| Inference latency | < 200ms |
| Collision rate | < 1% |

### TODO: Implement Evaluation Pipeline

In [ ]:
def evaluate_model(model: dict, eval_env, object_set: list,
                   num_episodes: int = 100) -> dict:
    """
    Evaluate the full model in simulation.

    Args:
        model: dict with 'encoder', 'rssm', 'flow_head'
        eval_env: PyBullet simulation environment with methods:
            - reset(object_name) -> (rgb, depth, instruction)
            - step(action) -> (rgb, depth, reward, done, info)
        object_set: list of object names to evaluate on
        num_episodes: number of episodes per object

    Returns:
        dict with:
            - "pick_success_rate": float
            - "mean_cycle_time": float (seconds)
            - "mean_smoothness": float (jerk metric)
            - "collision_rate": float
            - "per_object_success": dict mapping object name to success rate

    Implementation steps:
        1. For each object in object_set:
           a. Reset environment with that object
           b. Initialize RSSM state
           c. Loop until done or max 200 steps:
              - Encode observation
              - Update RSSM state (observe mode)
              - Build conditioning vector from RSSM state
              - Generate action chunk via flow_head.sample()
              - Execute first action from chunk in environment
              - Record metrics
           d. Log success/failure

        2. Aggregate metrics across all objects and episodes
        3. Print a summary table

    Hints:
        - Use action chunk overlap: generate a 16-step chunk but only
          execute the first 4 steps, then re-plan. This provides temporal
          consistency while allowing adaptation.
        - Compute jerk as the finite difference of acceleration:
          jerk = np.diff(actions, n=3, axis=0)
    """
    # YOUR CODE HERE
    raise NotImplementedError

## 3.7 Error Analysis

Systematically categorize and analyze failure cases from the evaluation. Understanding failure modes is critical for improving the system and for communicating results to stakeholders.

### TODO: Failure Mode Analysis

In [ ]:
def analyze_failures(eval_results: dict, model: dict, eval_env) -> dict:
    """
    Systematically categorize and analyze failure cases.

    From the evaluation results, identify all failed episodes and
    categorize them into failure modes.

    Expected failure categories:
        1. "grasp_slip" -- object was contacted but slipped during lift
        2. "miss" -- gripper did not contact the target object
        3. "wrong_object" -- grasped a different object than instructed
        4. "collision" -- collided with bin wall or other objects
        5. "timeout" -- exceeded maximum steps without completing task

    For each category:
        a. Count the number of failures
        b. Identify common object properties (size, shape, material)
           that correlate with this failure mode
        c. Visualize 3 example failures (save rgb frames at key moments)
        d. Suggest a specific mitigation strategy

    Returns:
        dict mapping failure category to:
            - "count": int
            - "fraction": float
            - "correlated_properties": list of strings
            - "mitigation": string describing fix
    """
    # YOUR CODE HERE
    raise NotImplementedError

### Thought Questions: Error Analysis

1. Which failure mode is most common? Is this expected given the model architecture?
2. Are there systematic differences in failure rates between suction and parallel-jaw grasps? Why?
3. How does failure rate correlate with object size? With transparency? With deformability?
4. If you could add ONE additional training signal to reduce failures, what would it be and why?

## 3.8 Scalability and Deployment Considerations

Before deploying this model in a real warehouse, we need to understand its inference performance characteristics and whether it meets the production latency targets.

### TODO: Inference Benchmarking

In [ ]:
def benchmark_inference(model: dict, device: torch.device,
                        num_warmup: int = 10, num_runs: int = 100) -> dict:
    """
    Profile inference latency for deployment feasibility.

    Measure the wall-clock time for each stage of the inference pipeline:
        1. Image preprocessing (resize, normalize)
        2. Observation encoding (DINOv2 + CLIP + cross-attention)
        3. RSSM state update
        4. Flow matching action generation (N=10 Euler steps)
        5. Total end-to-end latency

    Run each stage independently to isolate bottlenecks.

    Returns:
        dict with:
            - "preprocess_ms": mean preprocessing time in ms
            - "encode_ms": mean encoding time in ms
            - "rssm_ms": mean RSSM update time in ms
            - "flow_ms": mean flow matching time in ms
            - "total_ms": mean total time in ms
            - "throughput_hz": maximum achievable control frequency
            - "meets_latency_target": bool (total_ms < 200)

    Also compute:
        - Model parameter count per component
        - GPU memory usage during inference
        - Theoretical throughput on NVIDIA L4 vs T4 (Colab) vs A100

    Hints:
        - Use torch.cuda.synchronize() before timing to ensure accurate
          GPU measurements
        - Use torch.cuda.Event for precise GPU timing
        - Discard warmup runs (first 10) to avoid JIT compilation effects
        - Report mean, std, p50, p95, p99 latencies
    """
    # YOUR CODE HERE
    raise NotImplementedError

### Thought Questions: Scalability

1. Which component is the bottleneck? How would you optimize it for production?
2. What is the tradeoff between flow matching steps (quality) and latency? Plot this curve.
3. Could you use model distillation or quantization to meet the latency target on cheaper hardware?

## 3.9 Ethical and Regulatory Analysis

For each category below, provide 3-5 sentences of analysis.

### 1. Labor Displacement
- How many warehouse workers could this system displace at MegaMart?
- What is the timeline for displacement?
- What retraining or transition programs should NovaPick advocate for?
- What is the net job impact (jobs displaced vs. new jobs created in robotics maintenance, supervision, ML engineering)?

### 2. Safety
- What happens when the model makes a confident but wrong prediction?
- How should the system handle uncertainty? (When should it ask for human help vs. proceed?)
- What are the physical safety risks to humans working near these robots?
- How does ISO 10218-1 compliance affect the system design?

### 3. Bias and Fairness
- The training data comes from specific robot platforms in specific labs. What biases might this introduce?
- If the system performs worse on certain product categories (e.g., products marketed to specific demographics), is this a fairness concern?
- How would you audit the system for systematic performance disparities across product categories?

### 4. Environmental Impact
- Estimate the carbon footprint of training (128 A100s for 14 days)
- Compare to the carbon savings from more efficient warehouse operations
- Is the tradeoff justified?

### 5. Dual Use and Misuse
- Could this technology be misused for autonomous weapons or surveillance?
- What safeguards should NovaPick implement?

### TODO: Write Your Ethical Impact Assessment

In [ ]:
# YOUR ANALYSIS HERE
# Write your ethical impact assessment as a structured analysis (minimum 500 words total).
# Replace this cell with a markdown cell containing your analysis, organized by the
# five categories above.

## Summary

In this notebook, you built a simplified version of the NovaPick world model + VLA system for autonomous pick-and-place:

1. **Data (3.1-3.2):** Loaded and explored 15K robot manipulation episodes from DROID and Open X-Embodiment, implementing paired RGB-D augmentation that preserves spatial correspondence.

2. **Baseline (3.3):** Implemented a ResNet-18 + MLP grasp prediction baseline -- no language, no temporal reasoning, no world model. This establishes the performance floor.

3. **Model (3.4):** Built the three core components:
   - **ObservationEncoder** -- DINOv2 + CLIP with cross-attention fusion for grounded visual understanding
   - **RSSM** -- deterministic + stochastic state-space model for predicting grasp outcomes in latent space
   - **FlowMatchingActionHead** -- conditional flow model generating smooth 16-step action chunks

4. **Training (3.5):** Implemented the composite training loop with flow matching loss, RSSM KL loss, and grasp prediction loss, using AdamW with cosine LR schedule.

5. **Evaluation (3.6):** Tested the model in PyBullet simulation on both known and novel objects, measuring pick success rate, cycle time, smoothness, and collision rate.

6. **Error Analysis (3.7):** Categorized failures into 5 modes (grasp slip, miss, wrong object, collision, timeout) and identified object properties correlated with each failure type.

7. **Deployment (3.8):** Benchmarked inference latency per component to validate the 200ms latency target on an NVIDIA L4 GPU.

8. **Ethics (3.9):** Analyzed labor displacement, safety, bias, environmental impact, and dual-use concerns.

**Key takeaway:** The world model enables "mental rehearsal" -- predicting grasp outcomes before physical execution -- while the VLA architecture with flow matching generates smooth, language-conditioned trajectories that generalize zero-shot to novel objects. This combination transforms pick-and-place from a per-SKU engineering problem into a general-purpose learned capability.

For the full case study including production system design, see `case_study.md` in this directory.